IMPORTING DATA

In [1]:
%%time
#Import all the data
import pandas as pd
data = pd.read_csv('C:/Users/ritik/OneDrive/Desktop/ML_for_Automated_Essay_Grading-master/train_set_rel3.csv',encoding='latin-1')
data = data.dropna(axis=0, how='any')      #Drop NA values
df = data
df = data[['essay','domain1_score']]
df.columns = ['text','y'] #Take essays and the scores
df.head()

Wall time: 492 ms


,text,y
0,"Dear local newspaper, I think effects computer...",8.0
1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9.0
2,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7.0
3,"Dear Local Newspaper, @CAPS1 I have found that...",10.0
4,"Dear @LOCATION1, I know having computers has a...",8.0


In [2]:
%%time
#Remove the stopwords taken from two libraries - 'stop_words' and 'nltk'
essay_list = df['text']
from stop_words import get_stop_words
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words_list = set(get_stop_words('en')).union(set(stopwords.words('english')))  #Combining the stop words from both the packages

processed_essays = []
essay_list = list(essay_list)
for i in essay_list:
    temp = i
    parsed_essay = " ".join([word for word in temp.split() if word not in stop_words_list])
    processed_essays.append(parsed_essay)
df['text'] = processed_essays

Wall time: 1.01 s


C:\Users\ritik\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [3]:
%%time
#Lemmatize all the words in the essays
import nltk
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

s = ""
for i in range(len(processed_essays)):
    s = ""
    for word in processed_essays[i].split():
        s = s + lmtzr.lemmatize(word) + " "
    processed_essays[i] = s

Wall time: 6.14 s


In [4]:
%%time
#Remove Punctuation
import re
for i in range(len(processed_essays)):
    processed_essays[i] = re.sub(r'[^\w\s]','',processed_essays[i])

Wall time: 226 ms


In [5]:
%%time
#Append to the dataframe df

df['text'] = processed_essays

Wall time: 6.98 ms


C:\Users\ritik\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
%%time
#Compute the length of each essay - geometrical parameter

len_values = []
for i in range(0,len(df)):
    len_values.append(len(df.text.iloc[i]))

len_series = pd.Series(len_values)
df['length'] = len_series.values
df.head()

Wall time: 128 ms


C:\Users\ritik\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,text,y,length
0,Dear local newspaper I think effect computer p...,8.0,1147
1,Dear CAPS1 CAPS2 I believe using computer bene...,9.0,1516
2,Dear CAPS1 CAPS2 CAPS3 More people use compute...,7.0,995
3,Dear Local Newspaper CAPS1 I found many expert...,10.0,2180
4,Dear LOCATION1 I know computer positive effect...,8.0,1565


In [7]:
%%time
#Compute the no. of digits in each essay - geometrical parameter

digits_list = []

for i in range(0,len(df)):
    if(sum(c.isdigit() for c in df.text.iloc[i]) == 0):
        digits_list.append(0)
    else:
        digits_list.append(sum(c.isdigit() for c in df.text.iloc[i]))

digits_col = pd.Series(digits_list)
df['DIGITS'] = digits_col.values


Wall time: 1.7 s


C:\Users\ritik\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [8]:
%%time
#Count the parts of speech from each of the essay

import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
counts = []
for essay in processed_essays:
    tokenize = nltk.word_tokenize(essay)
    tagged = nltk.pos_tag(tokenize)
    counts.append(tagged)


Wall time: 1min 44s


In [9]:
%%time
#Convert the counter item to a dictionary and create a list of these dictionaries
from collections import Counter
pos_counter = []
for i in counts:
    a = Counter(tag for word,tag in i)
    pos_counter.append(dict(a))
df.head()

Wall time: 390 ms


,text,y,length,DIGITS
0,Dear local newspaper I think effect computer p...,8.0,1147,5
1,Dear CAPS1 CAPS2 I believe using computer bene...,9.0,1516,10
2,Dear CAPS1 CAPS2 CAPS3 More people use compute...,7.0,995,7
3,Dear Local Newspaper CAPS1 I found many expert...,10.0,2180,41
4,Dear LOCATION1 I know computer positive effect...,8.0,1565,4


In [10]:
%%time
#Append the word count according to each type of part of speech - append 0 where there is no words belonging to a specific type
#reference: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

orderedNames = ['CC','CD','DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT', 'POS', 'PRP'
               , 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'WDT', 'WP', 'WP$', 'WRB']
import numpy as np
pos_tag_count = []

for j in pos_counter:
    b = []
    for i in orderedNames:
        try:
            b.append(j[i])
        except KeyError:
            b.append(0)

            pos_tag_count.append(b)

Wall time: 196 ms


In [11]:
%%time
#Append the counts for each essay in the dataframe df
pos_tag_count = np.matrix(pos_tag_count)
for i in range(len(orderedNames)):
    df[orderedNames[i]] = pos_tag_count[:,i]

ValueError: Length of values (248210) does not match length of index (12977)

In [12]:
%%time
df

Wall time: 0 ns


,text,y,length,DIGITS
0,Dear local newspaper I think effect computer p...,8.0,1147,5
1,Dear CAPS1 CAPS2 I believe using computer bene...,9.0,1516,10
2,Dear CAPS1 CAPS2 CAPS3 More people use compute...,7.0,995,7
3,Dear Local Newspaper CAPS1 I found many expert...,10.0,2180,41
4,Dear LOCATION1 I know computer positive effect...,8.0,1565,4
...,...,...,...,...
12973,In story mother daughter either enemy friends ...,35.0,2756,80
12974,I never understood meaning laughter shortest d...,32.0,1706,32
12975,When laugh CAPS5 habit CAPS1 cause What cause ...,40.0,2939,57
12976,Trippin fence I NUM1 year young short NUM1 yea...,40.0,1952,20


In [13]:

%%time
#Compute a tfidf sparse matrix based on the processed_essays
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
#Create a sparse matrix
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
X_tfidf = vectorizer.fit_transform(processed_essays)

Wall time: 1.66 s


In [14]:
%%time
#Convert the tfidf sparse matrix to a dense matrix and append to a dataframe
l = pd.DataFrame(X_tfidf.todense())
print(l.head())

   0      1      2      3      4      5      6      7      8      9      ...  \
0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
1    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
2    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
3    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
4    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   

   43475  43476  43477  43478  43479  43480  43481  43482  43483  43484  
0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
1    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
2    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
3    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
4    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  

[5 rows x 43485 columns]
Wall time: 544 ms


In [15]:
%%time
print(df.head())

                                                text     y  length  DIGITS
0  Dear local newspaper I think effect computer p...   8.0    1147       5
1  Dear CAPS1 CAPS2 I believe using computer bene...   9.0    1516      10
2  Dear CAPS1 CAPS2 CAPS3 More people use compute...   7.0     995       7
3  Dear Local Newspaper CAPS1 I found many expert...  10.0    2180      41
4  Dear LOCATION1 I know computer positive effect...   8.0    1565       4
Wall time: 2.99 ms


In [16]:
%%time
#Reducing the dimensions of the matrix using PCA 
from sklearn.decomposition import PCA
pca = PCA(n_components=4)  # project data down to 4 dimensions
feature_vectors_pca2d = pca.fit_transform(X_tfidf.todense())
l = pd.DataFrame(feature_vectors_pca2d)
#Append the part of speech features, features from tfidf dense matrix and the geometrical features
result = pd.concat([df, l], axis=1)
result = result.dropna(axis=0, how='any')

Wall time: 30.5 s


In [17]:
%%time
print(result.head())

                                                text     y  length  DIGITS  \
0  Dear local newspaper I think effect computer p...   8.0  1147.0     5.0   
1  Dear CAPS1 CAPS2 I believe using computer bene...   9.0  1516.0    10.0   
2  Dear CAPS1 CAPS2 CAPS3 More people use compute...   7.0   995.0     7.0   
3  Dear Local Newspaper CAPS1 I found many expert...  10.0  2180.0    41.0   
4  Dear LOCATION1 I know computer positive effect...   8.0  1565.0     4.0   

          0         1         2         3  
0 -0.088093 -0.162042  0.018226 -0.043818  
1 -0.077345 -0.140638  0.040640 -0.013637  
2 -0.086067 -0.188553  0.041975 -0.052052  
3 -0.064251 -0.139934  0.011399 -0.033963  
4 -0.091227 -0.141933  0.052177 -0.037977  
Wall time: 23.5 ms


Although PCA assumes linearity, I am using PCA for dimensionality reduction here just to save the execution time. Using TSNE would be better but there is a trade off for time complexity.

In [18]:
%%time
X = result.iloc[:,2:]
y = result.iloc[:,1]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True,random_state=42)

Wall time: 24.2 ms


In [19]:
%%time
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics 
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
# Model Accuracy, how often is the classifier correct?
print("DecisionTreeClassifier\nAccuracy:",metrics.accuracy_score(y_test, y_pred))

DecisionTreeClassifier
Accuracy: 0.41756548536209553
Wall time: 486 ms


In [ ]:
%%time
#Import svm model
from sklearn import svm
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel
#Train the model using the training sets
clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
# Model Accuracy: how often is the classifier correct?
print("SVM\nAccuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier
#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
# prediction on test set
y_pred=clf.predict(X_test)
# Model Accuracy, how often is the classifier correct?
print("RandomForestClassifier\nAccuracy:",metrics.accuracy_score(y_test, y_pred))


##### I have tried multiple models and analyzed the accuracy for them. RandomForestClassifier gives the best accuracy than 'DecisionTreeClassifier', 'OneVsRestClassifier' and 'svm'

##### Hence I have decided to further fine tune the random forest classifier.



In [ ]:
%%time
#Creating a grid space with following hyperparameters
max_depth = [int(x) for x in np.linspace(10, 80, num = 5)]
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 150, num = 3)]
criterion = ["gini", "entropy"]
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'criterion' : criterion,
               
              }
print(random_grid)

In [ ]:
%%time
#Perform grid search with the grid above to tune the Random Forest Classifier
#reference: http://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import  GridSearchCV
clf = RandomForestClassifier()
rf_random =  GridSearchCV(estimator = clf, param_grid=random_grid)
rf_random.fit(X_train, y_train)
print(rf_random.best_params_)

In [ ]:
%%time
#Predict
model = rf_random.best_estimator_
y_pred = model.predict(X_test)

In [ ]:
%%time
#Model Evaluation
from sklearn.metrics import accuracy_score
a = accuracy_score(y_test, y_pred)
print("RandomForestClassifier")
print("Accuracy Score in % : ")
print(a*100)

In [ ]:
%%time
from sklearn.metrics import mean_squared_error
from math import sqrt
rms = sqrt(mean_squared_error(y_test, y_pred))
print("RMSE : " + str(rms))

Since there are about 60 classes for the prediction model, the ROC curve does not provide a good indsight towards the model performance.

Accuracy score which is coming out to be around 53.2%, is a good estimator of the model performance.

##### Most important features

In [ ]:
%%time
feat_imp = model.feature_importances_
#ind = np.argsort(feat_imp)[::-1]
#feat_imp[ind]
print(feat_imp)

In [ ]:
%%time
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

The most important feature among the feature space is the length feature which represents the length of the essay. There are many features which are some importance such as the use of certain words in the, digits and some specific parts of speech.